# Использование модели метчинга для поиска топ-5 услуг для новых услуг клиник

In [7]:
import pandas as pd
from matcher.matcher import SentenceSimilarity
from configs.config_matcher import MatcherConfig
from tensorflow.compat.v1.metrics import precision_at_top_k, recall_at_top_k
from tqdm import tqdm
tqdm.pandas()

In [8]:
data = pd.read_csv("data/data_for_matcher.csv")

Для проверки работы алгоритма, будет использоваться датасет из которого удалены дубликаты в столбце `local_name`

Параметры для конфига указаны в файле `configs/config_matcher.py`

In [9]:
config = MatcherConfig()
model = SentenceSimilarity(config=config)

https://github.com/benhamner/Metrics/blob/master/Python/ml_metrics/average_precision.py

In [10]:
import numpy as np

def apk(actual, predicted, k=10):
    """
    Computes the average precision at k.

    This function computes the average prescision at k between two lists of
    items.

    Parameters
    ----------
    actual : list
             A list of elements that are to be predicted (order doesn't matter)
    predicted : list
                A list of predicted elements (order does matter)
    k : int, optional
        The maximum number of predicted elements

    Returns
    -------
    score : double
            The average precision at k over the input lists

    """
    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    if not actual:
        return 0.0

    return score / min(len(actual), k)

def mapk(actual, predicted, k=10):
    """
    Computes the mean average precision at k.

    This function computes the mean average prescision at k between two lists
    of lists of items.

    Parameters
    ----------
    actual : list
             A list of lists of elements that are to be predicted 
             (order doesn't matter in the lists)
    predicted : list
                A list of lists of predicted elements
                (order matters in the lists)
    k : int, optional
        The maximum number of predicted elements

    Returns
    -------
    score : double
            The mean average precision at k over the input lists

    """
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])

In [ ]:
predict = data['service_name'].progress_apply(lambda x: model.get_top_k(x, top_k=5))

In [16]:
predict.apply(lambda x: [i['local_name'] for i in x]).values.tolist()

[['Выявление стресса и его стадии',
  'Комплексная оценка оксидативного стресса (7 параметров)',
  'Комплексное исследование, ОСМ',
  'Обследование домашнего персонала (расширенное)',
  'Комплексное исследование на гормоны (12 показателей)'],
 ['Клинический анализ крови: общий анализ, лейкоцитарная формула, СОЭ (с обязательной микроскопией мазка крови)',
  'Клинический анализ крови: общий анализ, лейкоцитарная формула, СОЭ (с микроскопией мазка крови при выявлении патологических изменений)',
  'Лейкоцитарная формула (с обязательной микроскопией мазка крови)',
  'Лейкоцитарная формула (с микроскопией мазка крови при выявлении патологических изменений)',
  'Клинический анализ крови с микроскопией лейкоцитарной формулы'],
 ['Лейкоцитарная формула (с обязательной микроскопией мазка крови)',
  'Лейкоцитарная формула (с микроскопией мазка крови при выявлении патологических изменений)',
  'Клинический анализ крови с микроскопией лейкоцитарной формулы',
  'Клинический анализ крови (c лейкоцита

In [22]:
mapk(data['local_name'].apply(lambda x: [x]).values.tolist(), predict.apply(lambda x: [i['local_name'] for i in x]).values.tolist(), k=5)

0.8989580759410262